Fork of  
[nog_3_3_inspect_gw.ipynb](nog_3_3_inspect_gw.ipynb)

In [ ]:
"""Iteration over outgoing edges, instead of nodes, see
https://nographs.readthedocs.io/en/latest/
graphs_and_adaptation.html#graphs-with-attributes"""
from functools import wraps
from decorator import decorator
import nographs as nog
from graphviz import Digraph
import onnx

In [ ]:
onnx_model = onnx.load("inception_v4_no_w.onnx")

In [ ]:
node_style ={"style": 'filled',            "shape": 'box',
             "align": 'left',              "fontsize": '12',
             "ranksep":'0.01',             "height":'0.02',   "width":'0.04',}
gr_style = {"linelength": '16', "rankdir": 'LR'}#"landscape":'True',

def adapter_graphviz( wrapped ):
    """translation to GraphViz visitor"""
    dot = Digraph( node_attr = node_style, graph_attr = gr_style, )
    @wraps( wrapped )
    def wrapper_bg( _edge, _trav ):
        dot.node( str(_edge), str(_edge) )
        for result in wrapped(_edge, _trav):
            edge = dot.edge( str(_edge), str(result[0]), str(result[1]), )
            yield result
    wrapper_bg.dot = dot
    return wrapper_bg

In [ ]:
stop_words = set(dir( object() ) + ['__getnewargs__'])#{'__getnewargs__', '__str__','__repr__','__doc__','__hash__'}

@decorator
def filter_loops( wrapped, _edge, _trav ):
    """Decorator for filtering, lets shrink graph a bit"""
    yield from filter( lambda e:e[0] != _edge, wrapped( _edge, _trav ) )

@decorator
def manage_unhashables( wrapped, _edge, _trav ):
    """Decorator for filtering, lets shrink graph a bit"""
    yield from filter( lambda e:getattr(e[0], '__hash__', None) is not None, wrapped( _edge, _trav ) )

@decorator
def filter_boring( wrapped, _edge, _trav ):
    """Decorator for filtering, lets shrink graph a bit"""
    yield from filter( lambda e:e[1] not in stop_words, wrapped( _edge, _trav ) )

@decorator
def find_mates( wrapped, _edge, _trav ):
    """filtering, in one or two steps we must return to same type"""
    for chi in wrapped( _edge, _trav ):
        if type(chi[0]) == type(_edge):
            yield chi
        else:
            yield from filter( lambda e:type(e[0]) == type(_edge), wrapped( chi, _trav ) )

In [ ]:
@adapter_graphviz
@filter_loops
#@find_mates
@filter_boring
@manage_unhashables
def all_attr(var, _):
    print('$')
    for attr_name, _ in zip(dir(var), range(50)):
        try:
            attr = getattr(var, attr_name)
            if hasattr( attr, "__call__"):
                # try:
                    attr = attr()
                # except Exception as ex:
                #     attr = str(ex)[:50]
            if attr is not None: #hasattr( attr, "__hash__"):
                yield ((attr), str(attr_name))
        except:
            continue

trav_forward = nog.TraversalBreadthFirst(next_labeled_edges=all_attr)
spoint = onnx_model.graph.input
trav_forward.start_from( spoint, build_paths=True )
for _, _ in zip(trav_forward, range(10)):
    pass

In [ ]:
all_attr.dot              # pylint: disable=pointless-statement

In [ ]:
all_attr.dot.attr( layout="neato")
all_attr.dot              # pylint: disable=pointless-statement

In [ ]:
set(dir( object() )) - set((  ))

In [ ]:
int(10).__floor__()